In [37]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from glob import glob
import torch
from torch.utils.data import Dataset
import torch.nn as nn
from torchvision import models
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights, efficientnet_b7, EfficientNet_B7_Weights
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir="runs/dfvd_b7")

In [38]:
df = pd.read_csv('../data/images/metadata34.csv')

df.head()

,filename,label,split,original
0,zhmjtavtpn.mp4,FAKE,train,ejnrxekehh.mp4
1,dazellwbsl.mp4,FAKE,train,xuxkfhqjiu.mp4
2,elwdztqnot.mp4,FAKE,train,xlyxbyhjsq.mp4
3,fiedwlcwyn.mp4,FAKE,train,skhnvzyukn.mp4
4,svbcewtjvh.mp4,FAKE,train,arrpqqfiah.mp4


In [39]:
df['filename'] = df['filename'].str.replace('.mp4', '', regex=False)
df['original'] = df['original'].fillna('').str.replace('.mp4', '', regex=False)

available_folders = set(os.listdir("../data/images"))
df['has_original'] = df['original'].apply(lambda x: x in available_folders if x else False)

df_filtered = df[(df['label'] == "FAKE") & (df['has_original'])]
df_filtered.head()

,filename,label,split,original,has_original
0,zhmjtavtpn,FAKE,train,ejnrxekehh,True
1,dazellwbsl,FAKE,train,xuxkfhqjiu,True
2,elwdztqnot,FAKE,train,xlyxbyhjsq,True
3,fiedwlcwyn,FAKE,train,skhnvzyukn,True
4,svbcewtjvh,FAKE,train,arrpqqfiah,True


In [40]:
fake_df = df[(df['label'] == 'FAKE') & (df['has_original'])].copy()

real_video_ids = fake_df['original'].unique()
real_df = df[df['filename'].isin(real_video_ids)].copy()

min_len = min(len(fake_df), len(real_df))
fake_df = fake_df.sample(min_len, random_state=42)
real_df = real_df.sample(min_len, random_state=42)

combined_df = pd.concat([fake_df, real_df], ignore_index=True)

In [41]:
video_ids = combined_df['filename'].unique()

train_ids, temp_ids = train_test_split(video_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.5, random_state=42)

def split_by_ids(df, ids):
    return df[df['filename'].isin(ids)].copy()

train_df = split_by_ids(combined_df, train_ids)
val_df = split_by_ids(combined_df, val_ids)
test_df = split_by_ids(combined_df, test_ids)


In [42]:
IMAGE_ROOT = "../data/images"

def get_image_label_pairs(df):
    pairs = []
    for _, row in df.iterrows():
        folder = os.path.join(IMAGE_ROOT, row['filename'])
        if not os.path.exists(folder):
            continue
        image_paths = glob(os.path.join(folder, "*.jpg"))
        for img in image_paths:
            pairs.append((img, row['label']))
    return pairs

train_data = get_image_label_pairs(train_df)
val_data = get_image_label_pairs(val_df)
test_data = get_image_label_pairs(test_df)

print("Train:", len(train_data), "Val:", len(val_data), "Test:", len(test_data))


Train: 5820 Val: 1250 Test: 1250


In [43]:
class DeepfakeDataset(Dataset):
    def __init__(self, image_label_list, transform=None):
        self.data = image_label_list
        self.transform = transform
        self.label_map = {"REAL": 0, "FAKE": 1}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"File error: {img_path}, ERROR: {e}")
            return self.__getitem__((idx + 1) % len(self))

        if self.transform:
            image = self.transform(image)

        label_id = self.label_map[label]
        return image, label_id



In [44]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

train_dataset = DeepfakeDataset(train_data, transform=transform)
val_dataset = DeepfakeDataset(val_data, transform=transform)
test_dataset = DeepfakeDataset(test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [45]:
def get_efficientnet_model(num_classes=2, freeze_base=True):
    weights = EfficientNet_B7_Weights.DEFAULT
    model = efficientnet_b7(weights=weights)

    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.5),
        nn.Linear(in_features, num_classes)
    )

    if freeze_base:
        for param in model.features.parameters():
            param.requires_grad = False

    return model


In [46]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0

    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()

    acc = correct / len(loader.dataset)
    avg_loss = total_loss / len(loader)
    return avg_loss, acc

def validate(model, loader, criterion, device, writer=None, epoch=None, show_confusion=False):
    model.eval()
    total_loss = 0
    y_true = []
    y_pred = []
    y_prob = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_prob.extend(probs.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    avg_loss = total_loss / len(loader)

    print(f"Val Accuracy: {acc:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

    if show_confusion:
        cm = confusion_matrix(y_true, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["REAL", "FAKE"])
        disp.plot(cmap="Blues")
        plt.title("Validation Confusion Matrix")

        fig = plt.gcf()
        if writer is not None and epoch is not None:
            writer.add_figure("Confusion Matrix", fig, global_step=epoch)

        plt.show()

    return avg_loss, acc, f1

In [47]:
def analyze_errors(model, loader, device):
    model.eval()
    y_true = []
    y_pred = []
    image_paths = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = outputs.argmax(dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            image_paths.extend([img_path for (img_path, _) in loader.dataset.data])

    fp_images = [img for img, yt, yp in zip(image_paths, y_true, y_pred) if yt == 0 and yp == 1][:3]
    fn_images = [img for img, yt, yp in zip(image_paths, y_true, y_pred) if yt == 1 and yp == 0][:3]

    def show_images(img_list, title):
        fig, axes = plt.subplots(1, len(img_list), figsize=(12, 4))
        fig.suptitle(title, fontsize=14)
        for i, img_path in enumerate(img_list):
            img = Image.open(img_path)
            axes[i].imshow(img)
            axes[i].axis("off")
            axes[i].set_title(Path(img_path).stem)
        plt.show()

    show_images(fp_images, "False Positives")
    show_images(fn_images, "False Negative")

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = get_efficientnet_model(num_classes=2, freeze_base=True).to(device)
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = CrossEntropyLoss()

best_val_acc = 0.0

EPOCHS = 50
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, f1_ = validate(model, val_loader, criterion, device, writer=writer, epoch=epoch, show_confusion=True)


    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Acc/train", train_acc, epoch)
    writer.add_scalar("Acc/val", val_acc, epoch)


    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), f"../models/b7/F1_{f1_}.pth")
        print("Model saved")


Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-c5b4e57e.pth" to C:\Users\hikme/.cache\torch\hub\checkpoints\efficientnet_b7_lukemelas-c5b4e57e.pth


100%|██████████| 255M/255M [00:23<00:00, 11.2MB/s] 
100%|██████████| 182/182 [06:29<00:00,  2.14s/it]


Val Accuracy: 0.6096, F1: 0.5933, AUC: 0.6611
Epoch 1/50
Train Loss: 0.6750, Acc: 0.5785
Val   Loss: 0.6638, Acc: 0.6096
Model saved


100%|██████████| 182/182 [07:46<00:00,  2.56s/it]


Val Accuracy: 0.6560, F1: 0.6504, AUC: 0.7029
Epoch 2/50
Train Loss: 0.6370, Acc: 0.6540
Val   Loss: 0.6371, Acc: 0.6560
Model saved


  1%|          | 1/182 [00:02<08:07,  2.69s/it]


KeyboardInterrupt: 

*In case the training logs are lost, I have documented all key validation results manually for reference.* 



| Epoch | Val Accuracy | F1 Score | AUC-ROC |
|-------|--------------|----------|---------|
| 3     | 0.6440       | 0.6792   | 0.7117  | 
| 4     | 0.6552       | 0.6879   | 0.7253  | 
| 5     | 0.6608       | 0.6958   | 0.7352  |
| 6     | 0.6632       | 0.6888   | 0.7368  | 
| 7     | 0.6712       | 0.6935   | 0.7495  | 
| 9     | 0.6784       | 0.6874   | 0.7619  | 
| 10    | 0.6832       | 0.7084   | 0.7635  | 
| 11    | 0.6992       | 0.7263   | 0.7700  | 
| 13    | 0.7064       | 0.7205   | 0.7749  | 
| 14    | 0.7048       | 0.7177   | 0.7812  |
| 15    | 0.7080       | 0.7229   | 0.7837  |             
| 16    | 0.7104       | 0.7270   | 0.7854  | 
| 17    | 0.7080       | 0.7298   | 0.7896  | 
| 18    | 0.7152       | 0.7343   | 0.7944  | 
| 20    | 0.7216       | 0.7387   | 0.7976  | 
| 23    | 0.7272       | 0.7446   | 0.8048  |             
| 25    | 0.7296       | 0.7447   | 0.8085  |             
| 27    | 0.7336       | 0.7513   | 0.8059  | 
| 29    | 0.7376       | 0.7515   | 0.8124  | 
| 33    | 0.7344       | 0.7533   | 0.8140  |             
| 36    | 0.7344       | 0.7511   | 0.8160  |             
| 41    | 0.7376       | 0.7449   | 0.8229  |             
| 46    | 0.7392       | 0.7631   | 0.8212  | 
| 47    | 0.7352       | 0.7589   | 0.8248  | 
